# Visualize Duckietown watchtower trajectory

### Importing and loading all the needed modules

In [8]:
import contracts
import yaml
import numpy as np
import geometry as geo
import numpy as np
from duckietown_world.world_duckietown.tile_template import load_tile_types
from duckietown_world.geo.measurements_utils import iterate_by_class
from duckietown_world.world_duckietown.tile import get_lane_poses
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_svg, ipython_draw_html
import duckietown_world as dw
from copy import deepcopy
import geometry as g
import pandas as pd
import sys
import traceback

contracts.disable_all()
m = dw.load_map('robotarium2')


INFO:dt-world:loading map robotarium2


### Setting up necessary classes and functions

In [6]:
class Person(dw.PlacedObject):
    
    def __init__(self, radius, *args, **kwargs):
        self.radius = radius
        dw.PlacedObject.__init__(self, *args, **kwargs)

    def draw_svg(self, drawing, g):
        # drawing is done using the library svgwrite
        c = drawing.circle(center=(0, 0), r=self.radius, fill='pink')
        g.add(c)
        # draws x,y axes
        dw.draw_axes(drawing, g)

    def extent_points(self):
        # set of points describing the boundary 
        L = self.radius
        return [(-L, -L), (+L, +L)]

def relative_pose(q0, q1):
    return g.SE2.multiply(g.SE2.inverse(q0), q1)

def interpolate(q0, q1, alpha):
    q1_from_q0 = relative_pose(q0, q1)
    vel = g.SE2.algebra_from_group(q1_from_q0)
    rel = g.SE2.group_from_algebra(vel * alpha)
    q = g.SE2.multiply(q0, rel)
    return q

### Importing trajectory files and extract relevant data

In [3]:
root = dw.PlacedObject()

realTimestamps = []
seqs2 = []
final_trajectory = []
startTime = 0

# Folder to trajectory for visualization
folderNamesSingle = 'autobot04_r2'

for fileName in range(0,500):
    trajectoryFile = 'trajectoryFiles/' + str(folderNamesSingle) + '/' + str(folderNamesSingle) + '_' + str(fileName) + '.yaml'

    try:
        with open(trajectoryFile, 'r') as stream:
            try:
                data = yaml.safe_load(stream)

            except yaml.YAMLError as exc:
                print(exc)
    except:
        if fileName !=0:
            break

    timestart = data['begin_time_stamp']
    data_points = len(data['trajectory_data'])

    x = np.zeros((data_points,))
    y= np.zeros((data_points,))
    R = np.zeros((3,3, data_points))            
    phi = np.zeros((3, data_points))

    dx = 999.999*np.ones((data_points, ))
    dy = 999.999*np.ones((data_points, ))

    dr = 999.999*np.ones((data_points, ))
    dphi = 999.999*np.ones((data_points, ))


    for idx, [time, traj] in enumerate(data['trajectory_data'].items()):

        x[idx] = np.array(traj[0])

        y[idx] = np.array(traj[1])

        R[:,:,idx] = np.reshape(np.array(traj[3:]), (3,3))


        phi[:,idx] = np.array([np.arctan2(-R[1,2,idx],R[2,2,idx]), 
                               np.arctan2(R[0,2,idx],np.sqrt(R[0,0,idx]**2 + R[0,1,idx]**2)),
                               np.arctan2(-R[0,1,idx], R[0,0,idx])])

        realTimestamps.append(float(time) + float(timestart))
        z = phi[2,idx]
        points = np.array([x[idx], y[idx]])
        final_trajectory.append([points, z])
    
final_array = final_trajectory

for entry in range(0, len(final_array)):
    x =  (final_array[entry][0][0] )  # -2.2
    y = final_array[entry][0][1] # + 0.8
    alpha = final_array[entry][1]
    q5 = geo.SE2_from_translation_angle([x,y],alpha)
    seqs2.append(q5)



trajectoryFiles/oliFiles/rec1_bright_lf2_clw/autobot05/autobot05_0.yaml
trajectoryFiles/oliFiles/rec1_bright_lf2_clw/autobot05/autobot05_1.yaml
415


### Calculate distance and relative heading to each timestamp

In [8]:
timestamps = range(len(seqs2)) # [0, 1, 2, ...]

# SE2Transform is the wrapper for SE2 used by Duckietown World 
transforms = [dw.SE2Transform.from_SE2(_) for _ in seqs2]
seq_me = dw.SampledSequence(timestamps, transforms)
counter = 0

for timestamp, pose_object in seq_me:
    print(counter)
    lanePoses = list(get_lane_poses(m, pose_object.as_SE2()))
    
    if len(lanePoses) == 0:
        print('')
        
    else:
        distance_from_center = lanePoses[0].lane_pose.distance_from_center
        rel_heading = lanePoses[0].lane_pose.relative_heading
        tile = list(lanePoses[0].tile.children.keys())[0]
        correctDir = lanePoses[0].lane_pose.correct_direction

    counter += 1
    print('')


0
1575567720.9999988 0.03282056665420652 0.0734862037090586 straight
 
1
1575567721.0666988 0.03452996838092859 0.07496031572183805 straight
 
2
1575567721.1332989 0.03623937010765155 0.07800900467327473 straight
 
3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: RuntimeWarning:

divide by zero encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: RuntimeWarning:

divide by zero encountered in double_scalars



1575567721.1999989 0.04136757528781951 0.0824436473883188 straight
 
4
1575567721.2666988 0.04307697701454247 0.08606609374740264 straight
 
5
1575567721.333299 0.043076977014542495 0.09170828895399379 straight
 
6
1575567721.399999 0.043076977014542495 0.10060996073148784 straight
 
7
1575567721.466699 0.044786378741265453 0.03127512873482267 straight
 
8
1575567721.5332987 0.044786378741265453 -0.041411945267611466 straight
 
9
1575567721.5999987 0.044786378741265453 -0.11889992135906635 straight
 
10
1575567721.6666987 0.041367575287819536 -0.17124629313657322 straight
 
11
1575567721.7332988 0.03623937010765155 -0.18053862906829443 straight
 
12
1575567721.7999988 0.03111116492748356 -0.19373013010670934 straight
 
13
1575567721.8666987 0.02598295974731557 -0.20410476257746257 straight
 
14
1575567721.9332988 0.020854754567147582 -0.1377350140943522 straight
 
15
1575567721.9999988 0.015726549386978733 -0.06879456713384764 straight
 
16
1575567722.0666988 0.017435951113701692 -0.06

lane2
1575567728.2666988 -0.12592658850992233 0.39838951503540054 curve_left
 
110
lane2
1575567728.333299 -0.1156890503235452 0.42128375255079864 curve_left
 
111
lane2
1575567728.399999 -0.10324978537476742 0.3933272736347214 curve_left
 
112
lane2
1575567728.466699 -0.08895051603627419 0.36666078790001905 curve_left
 
113
lane2
1575567728.5332987 -0.07402155949614873 0.3689612092478758 curve_left
 
114
lane2
1575567728.5999987 -0.059718652875887596 0.416507843757372 curve_left
 
115
lane2
1575567728.6666987 -0.05184131107360149 0.48854212281436027 curve_left
 
116
lane2
1575567728.7332988 -0.04353011933049877 0.5616037199309997 curve_left
 
117
lane2
1575567728.7999988 -0.04088307913140726 0.6582579350174649 curve_left
 
118
lane2
1575567728.8666987 -0.032169951983609876 0.7555369313762728 curve_left
 
119
lane2
1575567728.9332988 -0.02765689046047519 0.8686198594524288 curve_left
 
120
lane2
1575567728.9999988 -0.012405596956104575 0.893280757197713 curve_left
 
121
lane2
157556772

1575567735.400031 -0.21675204491615277 -0.5827717540162253 straight
 
216
1575567735.4666312 -0.22700845527648875 -0.6280265179612396 straight
 
217
1575567735.5333312 -0.23726486563682472 -0.6793808924330567 straight
 
218
1575567735.6000311 -0.2509400794506066 -0.7336646819949243 straight
 
219
1575567735.6666312 -0.2594870880842214 -0.7835384700222894 straight
 
220
1575567735.7333312 -0.2646152932643885 -0.8269385950148569 straight
 
221
1575567735.8000312 -0.2680340967178344 -0.8621028059740674 straight
 
222
1575567735.8666313 -0.26974349844455736 -0.8896162943691178 straight
 
223
1575567735.9333313 -0.2714529001712803 -0.9110985610956741 straight
 
224
1575567736.0000312 -0.2680340967178344 -0.9266092194812182 straight
 
225
1575567736.0666313 -0.26632469499111144 -0.9388194914845168 straight
 
226
1575567736.1333313 -0.25606828463077547 -0.9493890141386505 straight
 
227
1575567736.2000313 -0.23384606218338058 -0.9610337840249682 straight
 
228
1575567736.2666311 -0.2099144380

1575567742.8000312 0.14222221636772148 0.4805172290680753 straight
 
327
1575567742.8666313 0.1627350370883941 0.4269097066333558 straight
 
328
1575567742.9333313 0.19521366989612576 0.352815559027573 straight
 
329
1575567743.0000312 0.21059828543663028 0.2743079384141105 straight
 
330
1575567743.0666313 0.22085469579696654 0.19613453636272496 straight
 
331
1575567743.1333313 0.2259829009771347 0.11940350837468672 straight
 
332
1575567743.2000313 0.22940170443058017 0.04285319798797695 straight
 
333
1575567743.2666311 0.22769230270385743 -0.03349623172024562 straight
 
334
1575567743.333331 0.224273499250412 -0.1092072506262709 straight
 
335
1575567743.400031 0.22085469579696657 -0.18674424880187643 straight
 
336
1575567743.4666312 0.21401708889007567 -0.2671457698001718 straight
 
337
1575567743.5333312 0.20888888370990757 -0.35203722638426743 straight
 
338
1575567743.6000311 0.19863247334957126 -0.44245698655715543 straight
 
339
1575567743.6666312 0.18666666126251222 -0.467

### Draw trajectory

In [5]:
# Load map
m = dw.load_map('robotarium2')

m.set_object("emded", Person(0.1), ground_truth=seq_me)
ipython_draw_html(m);


INFO:dt-world:loading map robotarium2
INFO:dt-world:area: RectangularArea(pmin=[-0.155 -0.151],pmax=[ 6.01499996 11.06117927])
INFO:dt-world:Written SVG to out/ipython_draw_html/139718182020712/drawing.svg
INFO:dt-world:Written HTML to out/ipython_draw_html/139718182020712/drawing.html
